<a href="https://colab.research.google.com/github/webminst/TCC/blob/main/Qdrant_Vector_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Banco de Vetoriais**
Este é um exemplo bem simples da implementação de um banco de dados vetoriais

# 1. Instalação
A estrutura Sentence Transformers nos dá acesso a Large Language Models comuns que transformam dados brutos em embeddings.

In [ ]:
pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

Uma vez codificados, esses dados precisam ser mantidos em algum lugar. O Qdrant permite armazenar dados como embeddings. É possível também usar o Qdrant para executar consultas de pesquisa nesses dados, ou seja, podemos pedir ao mecanismo para nos dar respostas relevantes que vão muito além da correspondência de palavras-chave.

`Este tutorial requer qdrant-client versão 1.7.1 ou superior.`

In [ ]:
pip install -U qdrant-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.3/309.3 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.27.2 which is incompatible.
g

# Importar os modelos
Uma vez que os dois frameworks principais estejam definidos, precisamos especificar os modelos exatos que esse mecanismo usará.

In [ ]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


O framework Sentence Transformers contém muitos modelos de incorporação. No entanto, all-MiniLM-L6-v2 é o codificador mais rápido para este tutorial.

In [ ]:
encoder = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# 2. Adicione o conjunto de dados
all-MiniLM-L6-v2 codificará os dados fornecidos. Aqui listaremos alguns livros de ficção científica da biblioteca do Victor Ropke. Cada livro tem metadados, um nome, autor, ano de publicação e uma breve descrição.

In [ ]:
documents = [
    {
        "name": "A Máquina do Tempo",
        "description": "Um homem viaja no tempo e testemunha a evolução da humanidade.",
        "author": "H.G. Wells",
        "year": 1895,
    },
    {
        "name": "Ender's Game - O Jogo do Exterminador",
        "description": "Um jovem é treinado para se tornar um líder militar em uma guerra contra uma raça alienígena.",
        "author": "Orson Scott Card",
        "year": 1985,
    },
    {
        "name": "Admirável Mundo Novo",
        "description": "Uma sociedade distópica onde as pessoas são geneticamente modificadas e condicionadas para se conformarem a uma hierarquia social estrita.",
        "author": "Aldous Huxley",
        "year": 1932,
    },
    {
        "name": "O Guia do Mochileiro das Galáxias",
        "description": "Uma série cômica de ficção científica que segue as desventuras de um humano involuntário e seu amigo alienígena.",
        "author": "Douglas Adams",
        "year": 1979,
    },
    {
        "name": "Duna",
        "description": "Um planeta deserto é palco de intrigas políticas e lutas pelo poder.",
        "author": "Frank Herbert",
        "year": 1965,
    },
    {
        "name": "Fundação",
        "description": "Um matemático desenvolve uma ciência para prever o futuro da humanidade e trabalha para salvar a civilização do colapso.",
        "author": "Isaac Asimov",
        "year": 1951,
    },
    {
        "name": "Snow Crash",
        "description": "Um mundo futurista onde a internet evoluiu para um metaverso de realidade virtual.",
        "author": "Neal Stephenson",
        "year": 1992,
    },
    {
        "name": "Neuromancer",
        "description": "Um hacker é contratado para realizar um hack quase impossível e é puxado para uma teia de intrigas.",
        "author": "William Gibson",
        "year": 1984,
    },
    {
        "name": "A Guerra dos Mundos",
        "description": "Uma invasão marciana da Terra joga a humanidade no caos.",
        "author": "H.G. Wells",
        "year": 1898,
    },
    {
        "name": "Jogos Vorazes",
        "description": "Uma sociedade distópica onde adolescentes são forçados a lutar até a morte em um espetáculo televisivo.",
        "author": "Suzanne Collins",
        "year": 2008,
    },
    {
        "name": "O Enigma de Andrômeda",
        "description": "Um vírus mortal vindo do espaço sideral ameaça exterminar a humanidade.",
        "author": "Michael Crichton",
        "year": 1969,
    },
    {
        "name": "A Mão Esquerda da Escuridão",
        "description": "Um embaixador humano é enviado a um planeta onde os habitantes não têm gênero e podem mudar de gênero à vontade.",
        "author": "Ursula K. Le Guin",
        "year": 1969,
    },
    {
        "name": "O Problema dos Três Corpos",
        "description": "Os humanos encontram uma civilização alienígena que vive em um sistema moribundo.",
        "author": "Liu Cixin",
        "year": 2008,
    },
]

# 3. Defina o local de armazenamento
Precisamos dizer ao Qdrant onde armazenar os embeddings. Esta é uma demonstração básica, então vamos usar nosso computador local e sua memória como armazenamento temporário.

In [ ]:
client = QdrantClient(":memory:")

# 4. Crie uma coleção
Todos os dados no Qdrant são organizados por coleções. Neste caso, está armazenando livros, então estamos chamando isso de my_books.

In [ ]:
client.recreate_collection(
    collection_name="my_books",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),  # Vector size is defined by used model
        distance=models.Distance.COSINE,
    ),
)

<ipython-input-7-bdcce16ba4c5>:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

Use `recreate_collection` se estiver experimentando e executando o script várias vezes. Esta função tentará primeiro remover uma coleção existente com o mesmo nome.

O parâmetro `vector_size` define o tamanho dos vetores para uma coleção específica. Se o tamanho deles for diferente, é impossível calcular a distância entre eles. 384 é a dimensionalidade de saída do codificador. Você também pode usar `model.get_sentence_embedding_dimension()` para obter a dimensionalidade do modelo que está usando.

O parâmetro `distance` permite que você especifique a função usada para medir a distância entre dois pontos.

# 5. Carregar dados para coleção
Dizemos ao banco de dados para fazer upload `documents` para a coleção `my_books`. Isso dará a cada registro um id e um payload. O payload é apenas os metadados do conjunto de dados.

In [ ]:
client.upload_points(
    collection_name="my_books",
    points=[
        models.PointStruct(
            id=idx, vector=encoder.encode(doc["description"]).tolist(), payload=doc
        )
        for idx, doc in enumerate(documents)
    ],
)

# 6. Fazendo uma pergunta ao motor
Agora que os dados estão armazenados no Qdrant, podemos fazer perguntas e receber resultados semanticamente relevantes.

In [ ]:
hits = client.search(
    collection_name="my_books",
    query_vector=encoder.encode("Invasão alienígena").tolist(),
    limit=3,
)
for hit in hits:
    print(hit.payload, "score:", hit.score)

{'name': 'A Guerra dos Mundos', 'description': 'Uma invasão marciana da Terra joga a humanidade no caos.', 'author': 'H.G. Wells', 'year': 1898} score: 0.5398527268233044
{'name': 'O Guia do Mochileiro das Galáxias', 'description': 'Uma série cômica de ficção científica que segue as desventuras de um humano involuntário e seu amigo alienígena.', 'author': 'Douglas Adams', 'year': 1979} score: 0.45249660941013686
{'name': 'O Problema dos Três Corpos', 'description': 'Os humanos encontram uma civilização alienígena que vive em um sistema moribundo.', 'author': 'Liu Cixin', 'year': 2008} score: 0.43991695584818624


Resposta:

O mecanismo de busca mostra três das respostas mais prováveis ​​que têm a ver com a `invasão alienígena`. Cada uma das respostas recebe uma pontuação para mostrar o quão próxima a resposta está da consulta original.

# Restringir a consulta
Que tal o livro mais recente do início dos anos 2000?

In [ ]:
hits = client.search(
    collection_name="my_books",
    query_vector=encoder.encode("Invasão alienígena").tolist(),
    query_filter=models.Filter(
        must=[models.FieldCondition(key="year", range=models.Range(gte=2000))]
    ),
    limit=1,
)
for hit in hits:
    print(hit.payload, "score:", hit.score)

{'name': 'O Problema dos Três Corpos', 'description': 'Os humanos encontram uma civilização alienígena que vive em um sistema moribundo.', 'author': 'Liu Cixin', 'year': 2008} score: 0.43991695584818624
